In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re

stop_words = set(stopwords.words('english'))
regex_for_removing_numbers = "^[0-9]"


def tokenize(sentence):
    '''
    parse the sentence in to words
    :param sentence:
    :return:
    '''
    words = word_tokenize(sentence)
    return words


def tokenize_list_sentences(sentences):
    '''
    parse a list of sentence to words
    :param sentences: list of sentences
    :return:
    '''
    words = []
    for sentence in sentences:
        word = word_tokenize(sentence)
        words.append(word)
    return words


def stopwords_remove(words):
    '''
    remove stop-words from the list of words and return a filtered words
    :param words:
    :return:
    '''
    filtered_sentence = []
    for w in words:
        w = w.lower()
        # removes all the words starting with numbers
        match = re.search(regex_for_removing_numbers, w)
        if w not in stop_words and not match:
            # removes all special charecters
            new_word = ''.join(e for e in w if e.isalnum())
            if new_word != '':
                filtered_sentence.append(new_word)
    return filtered_sentence


def stemming(words):
    '''
    stem the verbs and adverbs into simple nouns
    :param words:
    :return:
    '''
    ps = PorterStemmer()
    stem_words = []
    for w in words:
        stem_words.append(ps.stem(w))
    return stem_words


def create_corpus(sentences):
    '''
    create a corpus/bag of words
    :param filepath:
    :return:
    '''
    corpus = set()
    for sentence in sentences:
        corpus.update(tokenize(sentence))

    print "corpus size:", corpus.__len__()
    filtered_stopwords = stopwords_remove(corpus)

    print "corpus size after removing stop words :", filtered_stopwords.__len__()
    filtered_stemming = stemming(filtered_stopwords)

    return filtered_stemming


def create_corpus_from_sentence(sentence):
    '''
    create a corpus with removed stop words and stemming.
    :param sentence:
    :return:
    '''
    corpus = tokenize(sentence)
    filtered_stopwords = stopwords_remove(corpus)

    filtered_stemming = stemming(filtered_stopwords)

    return filtered_stemming


In [ ]:
import pandas as pd

import corpora_generator

def freq_gen(corpus):
    return reduce(lambda d, c: d.update([(c, d.get(c, 0) + 1)]) or d, corpus, {})


def hashing_fun(desc):
    '''
    convert each description into a hash value
    :param desc:
    :return:
    '''
    desHashMap = {}
    for key in desc.keys():
        sentence = desc[key]
        corpus = corpora_generator.create_corpus_from_sentence(sentence)
        hashcode = hash(frozenset((freq_gen(corpus))))
        desHashMap[key] = hashcode
    return desHashMap


def df_convert_desc(df):
    '''
    convert the description features content to hash values
    :param df: dataframe
    :return:
    '''
    hashcodesmap = hashing_fun(df["description"])
    df["description"] = hashcodesmap

df=pd.read_json("/input/train.json")
df_convert_desc(df)

print df["description"]

